# 必ず使用する機能

## 重みの保存
学習したモデルの重みは保存する必要がある  

In [ ]:
torch.save(
{
    "epoch": epoch,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    },
    '/Path/to/chkp_path.pth'
)

## 重みの読み込み
途中まで学習させていた重みを保存しておけば、あとからそれを読み込んで使用できる  
それを使って推論するもよし、学習を再開するもよし

In [ ]:
chkp_path = '/Path/to/chkp_path'  
checkpoint = torch.load(chkp_path)  
model.load_state_dict(checkpoint['model_state_dict'])

# ディレクトリ構成図（一例）

```
.  
└── sample_dir  
    ├── model.py: モデルを定義するファイル（Unetなど）
    ├── dataset.py: データセットを定義するファイル（データの読み込み・前処理なども）
    ├── loss.py: 損失関数を定義するファイル
    ├── train.py: 訓練ファイル
    ├── plot_loss.py: 損失をプロットし、現在の訓練状況を確認するファイル
    ├── test.py: 実際にモデルを読み込んで推論精度を可視化するファイル
    ├── data
    │   ├── traindata: 訓練データ
    │   └── testdata: テストデータ
    └── listfiles
        ├── train.list: 訓練データ一覧
        └── testdata: テストデータ一覧
```

# バックエンドで回す方法
普通にターミナルで実行していると、そのターミナルを消した時に実行自体も止まってしまう  
- `実行コマンド &`: 実行を裏で回す
- **tmux**: 仮想端末を作成し、接続を切ったり作業を復旧したりできる
- **byobu**: tmuxの進化Ver

# 機械学習関連知識

## 学習率
機械学習モデルの訓練中にパラメータ（重み）を更新する際、どの程度の幅で変化させるかを決める重要なハイパーパラメータ  


In [ ]:
# 学習率
lr = 0.01

## 過学習
<img src='img/over_fitting.png'></img>  
引用元: [zeroone[G検定（AI・機械学習）用語集]](https://zero2one.jp/ai-word/overfitting-2/)
### 特徴
- モデルが訓練データの細かいノイズや例外にまで適応しすぎ、汎化性能（新しいデータに対する予測力）が低下すること
- 訓練データでの精度が非常に高い一方、テストデータでの精度が低くなること

### 防ぐ方法
- データを増やす
- 正則化 (Regularization)
- クロスバリデーション (Cross-Validation)
- モデルの単純化


## データ拡張 (Data Augmentation)
元の訓練データをさまざまな方法で変形・加工することで、データの量を人工的に増やし、モデルの汎化性能を向上させる手法

**画像の場合の手法**
- 回転: 画像を一定の角度で回転させる。
- 反転・フリップ: 左右や上下に画像を反転させる。
- ズーム: 拡大・縮小を行う。
- 平行移動: 画像を上下左右にずらす。
- 色調変化: 明るさやコントラスト、色相などを変える。

In [ ]:
# data_augmentation = Trueにすることデータアーギュメント
data_augmentation = True

# dataset.py
# augment関数内でデータ拡張を行っている。今回は水平反転と回転。さらに追加してもOK
def augment(self, img, gt):
    # 同じシードでランダム変換を適用
    seed = torch.randint(0, 2**32, (1,)).item()
    
    # ランダム水平反転
    torch.manual_seed(seed)
    img = self.random_flip(img)
    torch.manual_seed(seed)
    gt = self.random_flip(gt)
    
    # ランダム回転
    torch.manual_seed(seed)
    img = self.random_rotation(img)
    torch.manual_seed(seed)
    gt = self.random_rotation(gt)
    
    return img, gt

## 正則化
機械学習モデルの複雑さを抑えるために、モデルのパラメータに制約を加え、過学習を防ぐ手法  
例）L1正則化（Lasso）、L2正則化（Ridge）

In [ ]:
# weight_decayを入れることでL2正則化
torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

## ドロップアウト（Dropout）
正則化手法の一つで、訓練中にランダムに一部のノード（ニューロン）を無効にして学習を行う方法 

In [ ]:
# ドロップアウト
# pはドロップアウト率（0.2~0.5が一般的）
nn.Dropout(p=0.5)

# ヒント

## クロスエントロピー損失のクラスごとの重み（weight）
これを指定することで、各クラスごとの重みを指定することができる。  
最初の値から順番に'CLASSES'の値を示す。  
CLASSES = ['backgrounds','aeroplane','bicycle','bird','boat','bottle', 'bus','car' ,'cat','chair','cow', 'diningtable','dog','horse','motorbike','person', 'potted plant', 'sheep', 'sofa', 'train', 'monitor','unlabeld']

In [ ]:
weight = torch.tensor([0.3, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5])

## 損失関数の比率（dice_weight）

In [ ]:
dice_weight = 0.5

# util.py
# class DiceCrossEntropyLoss
# 複合損失の計算
total_loss = (1 - self.dice_weight) * ce_loss + self.dice_weight * dice_loss